# Scraping of articles from La Repubblica website

In [1]:
#I import the necessary packages to webscrape
import requests
from lxml.html import fromstring
import time
import pandas as pd

#I create a function to extract the information from the website
def fetch_article_html(url):
    response = requests.get(url)
    return response.text

#I create a function to extract the text of the article and to put together all of the paragraphs
def extract_article_text(html):
    tree = fromstring(html)
    article_text = tree.xpath("//div/p/text() | //div/p/a/text() | /div/p/strong/text()")
    return ' '.join(article_text)

#since this is only one page, I define the url
url =  "https://www.repubblica.it/dossier/politica/elezioni-politiche-settembre/"

#I create a list in which to store the information
articles_data = []

try:
    #I check if the website exists and I fetch the HTML source from the website
    response = requests.get(url)
    if response.status_code == 404:
        print("Page not found:", url)
    htmlsource = response.text

    tree = fromstring(htmlsource)

    #I find the links and titles
    article_links = tree.xpath("//article/div/h2/a/@href | //article/h2/a/@href")
    article_title = tree.xpath("//article/div/h2/a/text() | //article/h2/a/text()")

    #I extract the article texts
    for link, title in zip(article_links, article_title):
        try:
            article_html = fetch_article_html(link)

            #I extract the full article text
            article_text = extract_article_text(article_html)

            #I append the links, titles and text to the list created above
            articles_data.append({
                    'Link': link,
                    'Title': title,
                    'Text': article_text
            })

            #I delay of 1 second 
            time.sleep(1) 

        except requests.exceptions.RequestException as e:
            print("Connection error:", e)
except requests.exceptions.RequestException as e:
    print("Connection error:", e)

#I create a dataframe from the articles data dictionary, I ccreate a new column with the name of the news outlet 
#and I save it as csv file
articles_Repubblica = pd.DataFrame.from_dict(articles_data)
articles_Repubblica['News outlet'] = pd.Series('La Repubblica', index=articles_Repubblica.index)
articles_Repubblica.to_csv('articles_Repubblica.csv', index=False)

#I print the DataFrame
articles_Repubblica

,Link,Title,Text,News outlet
0,https://www.repubblica.it/politica/2022/09/26/...,Come sarà il nuovo Parlamento. Al centrodestra...,Con la vittoria netta del centrodestra la geog...,La Repubblica
1,https://www.repubblica.it/politica/2022/09/26/...,Liliana Segre presidente del Senato in attesa ...,"Anche se per pochi giorni, la senatrice a vita...",La Repubblica
2,https://www.repubblica.it/economia/2022/09/26/...,"Draghi pronto a collaborare, ma scrivere la ma...",Il governo uscente assicura supporto. Le scelt...,La Repubblica
3,https://www.repubblica.it/politica/2022/09/26/...,Meloni sarà premier? Chi lo decide e quando,"Cosa succederÃ da domani in avanti, a spoglio...",La Repubblica
4,https://www.repubblica.it/politica/2022/09/26/...,"Il centrodestra primo in 98 province su 105, c...",Il voto ridisegna la geografia politica. Il M5...,La Repubblica
...,...,...,...,...
191,https://video.repubblica.it/dossier/elezioni-2...,Salvini e Meloni tra baci e coltelli: il nuovo...,,La Repubblica
192,https://video.repubblica.it/edizione/bologna/b...,"Bologna, insulti e minacce ai militanti di Fra...",,La Repubblica
193,https://video.repubblica.it/dossier/elezioni-2...,"Pontida, i dubbi dei militanti leghisti sul fu...",C'è chi propone di aprire un congresso dopo le...,La Repubblica
194,https://video.repubblica.it/dossier/elezioni-2...,"A Monza Letta sfida la Lega con 500 sindaci: ""...","A 35 km dalla festa di Pontida, il Partito Dem...",La Repubblica


#I create a condition with a list of rows and then use it to extract those rows from the dataset and put into another dataset, 
#which will serve for the labeled dataset. Then, I drop those columns from the original dataset, as this will become the unlabeled dataset.I save both datasets and I reset their indexes 

In [2]:
condition = list(range(1, 15)) + list(range(85, 99)) + list(range(181, 195))
articlesRepubblica_labeled = articles_Repubblica.loc[condition].copy()
articlesRepubblica_labeled.to_csv('articlesRepubblica_labeled.csv', index=False)
articlesRepubblica_unlabeled = articles_Repubblica.drop(list(range(1, 15)) + list(range(85, 99)) + list(range(181, 195)))
articlesRepubblica_unlabeled.to_csv('articlesRepubblica_unlabeled.csv', index=False)
articlesRepubblica_labeled.reset_index(drop=True, inplace=True)
articlesRepubblica_unlabeled.reset_index(drop=True, inplace=True)

In [3]:
#I check that everything went smoothly
articlesRepubblica_unlabeled

,Link,Title,Text,News outlet
0,https://www.repubblica.it/politica/2022/09/26/...,Come sarà il nuovo Parlamento. Al centrodestra...,Con la vittoria netta del centrodestra la geog...,La Repubblica
1,https://milano.repubblica.it/cronaca/2022/09/2...,Fratelli d'Italia batte Lega nei luoghi simbol...,Fratelli d'Italia dilaga anche nelle roccafort...,La Repubblica
2,https://www.repubblica.it/politica/2022/09/26/...,"Matteo Salvini, il tramonto del Capitano: i tr...",La parabola discendente del leader della Lega....,La Repubblica
3,https://www.repubblica.it/politica/2022/09/26/...,"La Lega è vicina, ma a Berlusconi non riesce i...","PiÃ¹ che collasso, prove di consunzione. I pri...",La Repubblica
4,https://www.repubblica.it/politica/2022/09/26/...,"Giorgia Meloni, la storia politica e le radici...","Il concetto di famiglia, uscito intatto da un'...",La Repubblica
...,...,...,...,...
149,https://www.repubblica.it/politica/2022/09/18/...,Inchiesta su Silvio Berlusconi L'uomo che ha r...,"Il Berlusconismo Ã¨ finito, fagocitato dal sal...",La Repubblica
150,https://www.repubblica.it/politica/2022/09/23/...,"Premier di destra e di sinistra. Populista, so...","Premier di destra e di sinistra. Populista, so...",La Repubblica
151,https://www.repubblica.it/politica/2022/09/20/...,La zizzania corre su You Tube: le elezioni inq...,"Un testo da duemila battute, unâintera carte...",La Repubblica
152,https://www.repubblica.it/politica/2022/09/18/...,"L'appello dell' Anpi: ""Andate alle urne, un vo...",E' un appello a smuovere il primo partito in I...,La Repubblica


In [4]:
#I check that everything went smoothly
articlesRepubblica_labeled

,Link,Title,Text,News outlet
0,https://www.repubblica.it/politica/2022/09/26/...,Liliana Segre presidente del Senato in attesa ...,"Anche se per pochi giorni, la senatrice a vita...",La Repubblica
1,https://www.repubblica.it/economia/2022/09/26/...,"Draghi pronto a collaborare, ma scrivere la ma...",Il governo uscente assicura supporto. Le scelt...,La Repubblica
2,https://www.repubblica.it/politica/2022/09/26/...,Meloni sarà premier? Chi lo decide e quando,"Cosa succederÃ da domani in avanti, a spoglio...",La Repubblica
3,https://www.repubblica.it/politica/2022/09/26/...,"Il centrodestra primo in 98 province su 105, c...",Il voto ridisegna la geografia politica. Il M5...,La Repubblica
4,https://www.repubblica.it/politica/2022/09/26/...,"Per chi votano le donne, i laureati, i giovani...","Sembra un paradosso, dopo una compagna elettor...",La Repubblica
5,https://www.repubblica.it/politica/2022/09/26/...,Nel Nordest della Lega doppiata da FdI: “La ba...,"Dai Serenissimi ai ""patrioti"", dalle bandiere...",La Repubblica
6,https://www.repubblica.it/politica/2022/09/26/...,"La sconfitta del Pd nelle zone rosse, Ghisleri...",La sondaggista di Euromedia Research analizza ...,La Repubblica
7,https://www.repubblica.it/editoriali/2022/09/2...,Dove Giorgia Meloni porterà l'Italia in Europa,PoichÃ© FdI fa parte dello schieramento sovran...,La Repubblica
8,https://www.repubblica.it/commenti/2022/09/26/...,Ricominciare dal cuore,Letta da solo non ce la poteva fare contro la ...,La Repubblica
9,https://www.repubblica.it/cultura/2022/09/26/n...,La lunga strada che attende il Pd dopo la scon...,"Eâ saltato il tappo, e il tappo era il . Ch...",La Repubblica
